In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import pygeohash
import folium
import geopy.distance

In [2]:
crime_map_df = gpd.read_file('open_data/crime_map_data.geojson')
crime_map_df = crime_map_df.to_crs("epsg:4326")
crime_map_df.head()

,OBJECTID,CASE_NO,DATE,DESCRIPTION,LOCATION,CITY,Latitude,Longitude,GlobalID,geometry
0,3,202200349491,"Fri, 25 Nov 2022 21:33:48 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.350226397496606,-79.80152666827507,f623ed61-fbc7-4391-8038-9e385d6ef3dd,POINT (-79.80153 43.35023)
1,10,202200351471,"Sun, 27 Nov 2022 23:46:00 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.33830850583127,-79.7853229172281,aff64172-9cc4-44b0-a78b-71d0c21177c8,POINT (-79.78532 43.33831)
2,13,202200353214,"Tue, 29 Nov 2022 15:40:00 GMT",MVC - HIT & RUN,GUELPH LI,BURLINGTON,43.3646561940604,-79.8220414963383,8883b2d6-9c65-4f61-a25d-5e9e3d77d4d9,POINT (-79.82204 43.36466)
3,21,202200357807,"Sun, 04 Dec 2022 17:44:48 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.34732873413244,-79.79743701462455,48573cd9-b167-4c66-8bc7-7d6e32f546ee,POINT (-79.79744 43.34733)
4,24,202200361004,"Thu, 08 Dec 2022 00:04:00 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.352177632465434,-79.80386318857246,6c97ac07-1a63-41b5-95f8-725dea1811f4,POINT (-79.80386 43.35218)


In [3]:
road_segments_df = gpd.read_file('open_data/burlington_road_segments.geojson')
road_segments_df = road_segments_df.to_crs("epsg:4326")
road_segments_df['SPEEDLIMIT'] = road_segments_df['SPEEDLIMIT'].fillna(50) #unless posted otherwise, the default speed limit in Burlington is 50km/h. We'll use that to fill NaN values
road_segments_df['ACTUAL_WIDTH'] = road_segments_df['ACTUAL_WIDTH'].fillna(road_segments_df['DEEMED_WIDTH'])
road_segments_df['ACTUAL_WIDTH'] = road_segments_df['ACTUAL_WIDTH'].where((road_segments_df['ACTUAL_WIDTH'] < 9990) & (road_segments_df['ACTUAL_WIDTH'] > 0), road_segments_df['DEEMED_WIDTH'])
road_segments_df['WIDTH_PER_LANE'] = road_segments_df['ACTUAL_WIDTH']/road_segments_df['LANES']
road_segments_df.head()

,OBJECTID,STREET_NAME,STREET_TYPE,ALIAS_ST_N,L_LADD,L_HADD,R_LADD,R_HADD,STREET_CLASS,DEEMED_WIDTH,...,RIGHT_COMMUNITY,AVLZONE_RPM,ROADDIRECTION,LAST_EDITED_DATE,WC_ZONE,ACTUAL_WIDTH,AREA40KM,SHAPELEN,geometry,WIDTH_PER_LANE
0,935288,GLENWOOD AVE.,MINOR,None,800,914,775,905,U,20.0,...,BURLINGTON,South West,None,2023-02-08T20:51:59,1.0,20.0,N,623.313576,"LINESTRING (-79.83936 43.30586, -79.83976 43.3...",10.0
1,935289,FAIRVIEW ST.,MAJOR,None,0,0,0,0,U,36.0,...,BURLINGTON,South East,None,2023-02-08T20:52:22,15.0,36.0,N,121.952115,"LINESTRING (-79.79247 43.35106, -79.79186 43.3...",7.2
2,935290,SHARALIN CRT.,MINOR,None,569,589,570,586,U,20.0,...,BURLINGTON,South West,None,2023-02-08T20:51:49,3.0,20.0,N,67.724165,"LINESTRING (-79.82544 43.31448, -79.82485 43.3...",10.0
3,935291,WALKER'S LINE,MAJOR,None,476,534,495,515,U,35.0,...,BURLINGTON,South East,None,2023-02-08T20:51:59,9999.0,35.0,N,353.787548,"LINESTRING (-79.76953 43.35284, -79.77167 43.3...",7.0
4,935292,UPLAND DR.,MINOR,None,0,0,0,0,U,20.0,...,BURLINGTON,North East,None,2023-02-08T20:52:10,18.0,20.0,N,47.203260,"LINESTRING (-79.82332 43.36800, -79.82342 43.3...",10.0


In [4]:
road_segments_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4456 entries, 0 to 4455
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   OBJECTID          4456 non-null   int64   
 1   STREET_NAME       4362 non-null   object  
 2   STREET_TYPE       4456 non-null   object  
 3   ALIAS_ST_N        235 non-null    object  
 4   L_LADD            4456 non-null   object  
 5   L_HADD            4456 non-null   object  
 6   R_LADD            4456 non-null   object  
 7   R_HADD            4456 non-null   object  
 8   STREET_CLASS      4455 non-null   object  
 9   DEEMED_WIDTH      4456 non-null   float64 
 10  W_CONTROL         4225 non-null   object  
 11  OWNER             4456 non-null   object  
 12  LANES             4453 non-null   float64 
 13  SPEEDLIMIT        4456 non-null   float64 
 14  LEFT_COMMUNITY    4456 non-null   object  
 15  RIGHT_COMMUNITY   4456 non-null   object  
 16  AVLZONE_RPM     

In [5]:
road_segments_df.columns

Index(['OBJECTID', 'STREET_NAME', 'STREET_TYPE', 'ALIAS_ST_N', 'L_LADD',
       'L_HADD', 'R_LADD', 'R_HADD', 'STREET_CLASS', 'DEEMED_WIDTH',
       'W_CONTROL', 'OWNER', 'LANES', 'SPEEDLIMIT', 'LEFT_COMMUNITY',
       'RIGHT_COMMUNITY', 'AVLZONE_RPM', 'ROADDIRECTION', 'LAST_EDITED_DATE',
       'WC_ZONE', 'ACTUAL_WIDTH', 'AREA40KM', 'SHAPELEN', 'geometry',
       'WIDTH_PER_LANE'],
      dtype='object')

In [6]:
intersections_df = gpd.read_file('open_data/burlington_intersections.geojson')
intersections_df = intersections_df.to_crs("epsg:4326")
intersections_df.head()

,OBJECTID,STREET1,STREET2,STREET1_DIR,STREET2_DIR,GLOBALID,geometry
0,469884,GLENCREST RD,ROSEDALE CRES,North-South,East-West,{82F4F783-773B-4152-BB97-383DC537905A},POINT (-79.78455 43.34352)
1,469885,WOODWARD AVE,OXFORD RD,East-West,North-South,{FD42AE0E-B476-49C3-8382-AF4C2045F9F0},POINT (-79.79040 43.34314)
2,470078,FASSEL AVE,PHYLLIS ST,East-West,North-South,{90E41D6B-22B9-484B-9BCF-948505885848},POINT (-79.80764 43.34322)
3,470079,GLENCREST RD,ARLINGTON BLVD,East-West,North-South,{4F9ADBC7-B942-464A-9C57-F98ACF3B9954},POINT (-79.78529 43.34296)
4,470080,NEW ST,DYNES RD,East-West,North-South,{390FDEDB-E9FB-470B-86C8-CBCFD4FD70B8},POINT (-79.78038 43.34287)


## Geohash Conversion
We'll be using geohash level 8 as it's the geohash level at 20 metres. This should be small enough for any inaccuracies in the road segments. For segments, we'll grab the geohash of each end of the line string. 

In [7]:
intersections_df['geohash'] = intersections_df['geometry'].apply(lambda x: pygeohash.encode(x.x, x.y, precision=8)) 
crime_map_df['geohash'] = crime_map_df['geometry'].apply(lambda x: pygeohash.encode(x.x, x.y, precision=8)) 

In [8]:
def segment_geohash(line, which = 0):
    assert which == 0 or which == -1
    pos = line.coords[which]
    return pygeohash.encode(pos[0], pos[1], precision=8)
road_segments_df['geohash_1'] = road_segments_df['geometry'].apply(lambda x: segment_geohash(x, 0))
road_segments_df['geohash_2'] = road_segments_df['geometry'].apply(lambda x: segment_geohash(x, -1))
del segment_geohash

## Getting Intersection Info
Now only keep the needed info for the road segments 

In [9]:
segment_info_df = road_segments_df[['STREET_NAME', 'STREET_TYPE', 'STREET_CLASS', 'ACTUAL_WIDTH', 'SPEEDLIMIT', 'LANES', 'OWNER', 'geohash_1', 'geohash_2', 'WIDTH_PER_LANE']]#.query("OWNER != 'MTO'")
segment_info_df = pd.concat(
    [segment_info_df.rename({'geohash_1': 'geohash'}, axis=1).drop('geohash_2', axis=1),
    segment_info_df.rename({'geohash_2': 'geohash'}, axis=1).drop('geohash_1', axis=1)],
    ignore_index=True
)
segment_info_df = segment_info_df.groupby('geohash')

def set_append(items):   
    items = list(set(items))
    items.sort()
    print(items)
    return ' & '.join(items)
segment_info_df = segment_info_df.agg(
    street_names=pd.NamedAgg(column='STREET_NAME', aggfunc=lambda x: list(set(x))),
    street_types=pd.NamedAgg(column='STREET_TYPE', aggfunc=lambda x: list(set(x))),
    owners=pd.NamedAgg(column='OWNER', aggfunc=lambda x: list(set(x))),
    max_width=pd.NamedAgg(column='ACTUAL_WIDTH', aggfunc='max'),
    min_width=pd.NamedAgg(column='ACTUAL_WIDTH', aggfunc='min'),
    max_speed_limit=pd.NamedAgg(column='SPEEDLIMIT', aggfunc='max'),
    min_speed_limit=pd.NamedAgg(column='SPEEDLIMIT', aggfunc='min'),
    max_lanes=pd.NamedAgg(column='LANES', aggfunc='max'),
    min_lanes=pd.NamedAgg(column='LANES', aggfunc='min'),
    max_width_per_lane=pd.NamedAgg(column='WIDTH_PER_LANE', aggfunc='max'),
    min_width_per_lane=pd.NamedAgg(column='WIDTH_PER_LANE', aggfunc='min')
)
del set_append
segment_info_df = segment_info_df.reset_index()
segment_info_df.head(n=10)

,geohash,street_names,street_types,owners,max_width,min_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes,max_width_per_lane,min_width_per_lane
0,hcyc3e6y,[NO. 6 HWY. N],[BOTH],[MTO],20.0,20.0,80.0,80.0,0.0,0.0,inf,inf
1,hcyc3e6z,[NO. 6 HWY. N],[MAJOR],[MTO],20.0,20.0,80.0,80.0,0.0,0.0,inf,inf
2,hcyc3ect,"[NO. 6 HWY. N, MOUNTAIN BROW RD. W]","[MAJOR, MINOR]","[COB, MTO]",20.0,20.0,80.0,50.0,2.0,0.0,inf,10.0
3,hcyc3kyw,[NO. 6 HWY. N],"[MAJOR, BOTH]",[MTO],20.0,20.0,80.0,50.0,0.0,0.0,inf,inf
4,hcyc3m9y,[OLD YORK RD.],[MINOR],[COH],20.0,20.0,50.0,50.0,2.0,2.0,10.0,10.0
5,hcyc3mgh,"[NO. 6 HWY. N, OLD YORK RD.]","[BOTH, MINOR]","[COH, COB, MTO]",20.0,20.0,50.0,50.0,2.0,0.0,inf,10.0
6,hcyc3mgz,[NO. 6 HWY. N],[BOTH],[MTO],20.0,20.0,80.0,80.0,0.0,0.0,inf,inf
7,hcyc3mrv,[OLD YORK RD.],[MINOR],[COB],10.0,10.0,50.0,50.0,2.0,2.0,5.0,5.0
8,hcyc3msc,[NO. 6 HWY. N],[BOTH],[MTO],20.0,20.0,80.0,50.0,0.0,0.0,inf,inf
9,hcyc3muh,[NO. 6 HWY. N],[BOTH],[MTO],20.0,20.0,80.0,50.0,0.0,0.0,inf,inf


Next we match the segment information to the intersections

In [10]:
intersections_df

,OBJECTID,STREET1,STREET2,STREET1_DIR,STREET2_DIR,GLOBALID,geometry,geohash
0,469884,GLENCREST RD,ROSEDALE CRES,North-South,East-West,{82F4F783-773B-4152-BB97-383DC537905A},POINT (-79.78455 43.34352),hcyf479v
1,469885,WOODWARD AVE,OXFORD RD,East-West,North-South,{FD42AE0E-B476-49C3-8382-AF4C2045F9F0},POINT (-79.79040 43.34314),hcyf469e
2,470078,FASSEL AVE,PHYLLIS ST,East-West,North-South,{90E41D6B-22B9-484B-9BCF-948505885848},POINT (-79.80764 43.34322),hcycfr3w
3,470079,GLENCREST RD,ARLINGTON BLVD,East-West,North-South,{4F9ADBC7-B942-464A-9C57-F98ACF3B9954},POINT (-79.78529 43.34296),hcyf4792
4,470080,NEW ST,DYNES RD,East-West,North-South,{390FDEDB-E9FB-470B-86C8-CBCFD4FD70B8},POINT (-79.78038 43.34287),hcyf4k3m
...,...,...,...,...,...,...,...,...
2373,472449,RAVEN AVE,EAGLE DR,East-West,North-South,{33DCE4C9-E9D9-4382-970F-294AAE9D5F96},POINT (-79.83593 43.31142),hcyccd63
2374,472450,GO STATION ACCESS,WATERDOWN RD,East-West,North-South,{2225EDF3-5F76-49D9-97E2-FEA278109DCA},POINT (-79.85886 43.31140),hcyc9w4k
2375,472451,TEAL DR,CRANSTON CRT,East-West,North-South,{35D40AD8-F60D-4206-ADE2-1459D21AB74A},POINT (-79.83760 43.30918),hcyccd0b
2376,472452,NORTH SHORE BLVD E,FOREST GLEN AVE,East-West,North-South,{B0E94127-EE12-4CBF-8C86-F155F06EF6A3},POINT (-79.83805 43.30635),hcycc3yt


In [11]:
intersections_info_df = pd.merge(intersections_df[['geohash', 'STREET1', 'STREET2']], segment_info_df, how='left', on=['geohash'])
intersections_info_df.columns = map(str.lower, intersections_info_df.columns)
intersections_info_df= intersections_info_df.dropna(subset=['owners'])
intersections_info_df['min_speed_limit'] = intersections_info_df['min_speed_limit'].astype('int32')
intersections_info_df['max_speed_limit'] = intersections_info_df['max_speed_limit'].astype('int32')
intersections_info_df = intersections_info_df.drop_duplicates(subset=['geohash']) # there are some duplicates where multiple different road names meet, or intersections are counted twice
intersections_info_df.head()

,geohash,street1,street2,street_names,street_types,owners,max_width,min_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes,max_width_per_lane,min_width_per_lane
0,hcyf479v,GLENCREST RD,ROSEDALE CRES,"[ROSEDALE CRES., GLENCREST RD.]",[MINOR],[COB],20.0,20.0,40,40,2.0,2.0,10.0,10.00
1,hcyf469e,WOODWARD AVE,OXFORD RD,"[WOODWARD AVE., OXFORD RD.]",[MINOR],[COB],20.0,20.0,50,40,2.0,2.0,10.0,10.00
2,hcycfr3w,FASSEL AVE,PHYLLIS ST,"[PHYLLIS ST., FASSEL AVE.]",[MINOR],[COB],20.0,20.0,50,50,2.0,2.0,10.0,10.00
3,hcyf4792,GLENCREST RD,ARLINGTON BLVD,"[ARLINGTON BLVD., GLENCREST RD.]",[MINOR],[COB],20.0,20.0,40,40,2.0,2.0,10.0,10.00
4,hcyf4k3m,NEW ST,DYNES RD,"[DYNES RD., NEW ST.]","[MAJOR, MINOR]",[COB],35.0,20.0,60,40,4.0,2.0,10.0,8.75


Further analysis later on has shown that some road intersections do not exist in the dataset. Thusly we'll need to use the Road Segments to create intersections.

In [12]:
def is_already_intersection(row):
    return row['geohash'] not in intersections_info_df.geohash.values
valid_owners = ['TOO', 'TOM', 'COH', 'ROH', 'COB', 'PRI', 'MTO']
tdf = segment_info_df[segment_info_df.apply(is_already_intersection, axis=1)]
tdf = tdf[tdf['street_names'].apply(lambda x: len(x) >= 2)]
tdf['position'] = tdf['geohash'].apply(pygeohash.decode_exactly).apply(lambda x: (x[1], x[0]))
tdf['street1'] = tdf['street_names'].apply(lambda x: x[0])
tdf['street2'] = tdf['street_names'].apply(lambda x: x[1])
tdf = tdf[tdf['owners'].apply(lambda x: sum([1 if y in valid_owners else 0 for y in x]) == len(x))]
tdf = tdf.dropna(subset=['max_speed_limit'])
excluded_geohashes = [
    'hcyccuwv', # The Bridge Over Fairview from the mall    
    'hcycfj51' # Glendor Ave and Plains Road don't meet, but the points are just really close    
    'hcyf1fhe', # Old Lakeshore ends at Pearl, and Lakeshore at Pearl is already accounted for
    'hcycc9hs', # Lindley Common is way too close to the existing Plains Road intersection
]
tdf = tdf[tdf['geohash'].apply(lambda x: x not in excluded_geohashes)]
intersections_info_df = pd.concat([intersections_info_df, tdf.drop(columns=['position', 'street_names'])], ignore_index=True)
del is_already_intersection
intersections_info_df.head()

,geohash,street1,street2,street_names,street_types,owners,max_width,min_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes,max_width_per_lane,min_width_per_lane
0,hcyf479v,GLENCREST RD,ROSEDALE CRES,"[ROSEDALE CRES., GLENCREST RD.]",[MINOR],[COB],20.0,20.0,40.0,40.0,2.0,2.0,10.0,10.00
1,hcyf469e,WOODWARD AVE,OXFORD RD,"[WOODWARD AVE., OXFORD RD.]",[MINOR],[COB],20.0,20.0,50.0,40.0,2.0,2.0,10.0,10.00
2,hcycfr3w,FASSEL AVE,PHYLLIS ST,"[PHYLLIS ST., FASSEL AVE.]",[MINOR],[COB],20.0,20.0,50.0,50.0,2.0,2.0,10.0,10.00
3,hcyf4792,GLENCREST RD,ARLINGTON BLVD,"[ARLINGTON BLVD., GLENCREST RD.]",[MINOR],[COB],20.0,20.0,40.0,40.0,2.0,2.0,10.0,10.00
4,hcyf4k3m,NEW ST,DYNES RD,"[DYNES RD., NEW ST.]","[MAJOR, MINOR]",[COB],35.0,20.0,60.0,40.0,4.0,2.0,10.0,8.75


You can execute the code in the below cell to see a map of the added intersections.

In [13]:
position = (43.43228422732222, -79.90534169899263)
m = folium.Map(location=position, zoom_start=12)
for index, row in tdf.iterrows():    
    folium.Marker(location=row['position'], popup=str(row)).add_to(m)
del position
m

In [14]:
del tdf
m = 0
del m

## Export 1

In [15]:
crime_map_df.columns = map(str.lower, crime_map_df.columns)
crime_df = pd.DataFrame(crime_map_df.drop(columns='geometry'))
crime_df.to_json('prepared_data/crash_data.json', orient='records')

In [16]:
intersections_info_df.to_json('prepared_data/intersection_info.json', orient='records')

## Join Crimes to Intersections

In [17]:
data_df = pd.merge(crime_df, intersections_info_df, how='left', on=['geohash'])
data_df.head()

,objectid,case_no,date,description,location,city,latitude,longitude,globalid,geohash,...,street_types,owners,max_width,min_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes,max_width_per_lane,min_width_per_lane
0,3,202200349491,"Fri, 25 Nov 2022 21:33:48 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.350226397496606,-79.80152666827507,f623ed61-fbc7-4391-8038-9e385d6ef3dd,hcyf42wd,...,"[MAJOR, MINOR]","[COB, ROH]",42.0,20.0,60.0,50.0,8.0,5.0,6.400000,3.333333
1,10,202200351471,"Sun, 27 Nov 2022 23:46:00 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.33830850583127,-79.7853229172281,aff64172-9cc4-44b0-a78b-71d0c21177c8,hcyf45tb,...,[MAJOR],[COB],35.0,24.0,60.0,50.0,6.0,5.0,7.000000,4.000000
2,13,202200353214,"Tue, 29 Nov 2022 15:40:00 GMT",MVC - HIT & RUN,GUELPH LI,BURLINGTON,43.3646561940604,-79.8220414963383,8883b2d6-9c65-4f61-a25d-5e9e3d77d4d9,hcycfuc4,...,[MAJOR],"[ROH, COB]",42.0,40.0,60.0,60.0,6.0,5.0,8.400000,7.000000
3,21,202200357807,"Sun, 04 Dec 2022 17:44:48 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.34732873413244,-79.79743701462455,48573cd9-b167-4c66-8bc7-7d6e32f546ee,hcyf43kd,...,[MAJOR],"[ROH, COB]",42.0,40.0,60.0,50.0,9.0,7.0,5.714286,4.444444
4,24,202200361004,"Thu, 08 Dec 2022 00:04:00 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.352177632465434,-79.80386318857246,6c97ac07-1a63-41b5-95f8-725dea1811f4,hcyf480h,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
print("Non Matches: " + str(data_df[(data_df['street1'].isna()) & data_df['street2'].isna() & (data_df['street_names'].isna())].count()[0]))

Non Matches: 213


Looks like some points aren't properly matched. Gonna have to use the distance between the actual positions. Initially assumed that 35 metres is the furthest the intersection will be. However, there are still 4 cases that go unmatched. Looking at each one, it's likely safe to match them to their closest points. 

In [19]:
# max_distance_allowed = 0.035

unmatched_df = crime_map_df[(data_df['street1'].isna()) & data_df['street2'].isna() & (data_df['street_names'].isna())]
tdf = intersections_info_df[['geohash']].copy()
tdf['geometry'] = tdf['geohash'].apply(pygeohash.decode_exactly).apply(lambda x: (x[0], x[1]))
tdf['geohash_5'] = tdf['geohash'].apply(lambda x: x[:5]) # geohash 5 covers +/-2.4 km
geohash_transform = dict()
bad_count = 0
for index, row in unmatched_df.iterrows():    
    if (row['geohash'] in geohash_transform.keys()): continue
    pos = (row['latitude'], row['longitude'])
    geohash_5 = row['geohash'][:5]
    other_df = tdf#tdf[tdf['geohash_5'] == geohash_5]
    min_dist = 100000
    min_dist_found = 10000
    not_chosen_winner = None
    winner = None
    for j, orow in other_df.iterrows():
        opos = (orow['geometry'][1], orow['geometry'][0])
        dist = geopy.distance.geodesic(pos, opos, ellipsoid='WGS-84').km
        if dist < min_dist_found:
            min_dist_found = dist
            not_chosen_winner = orow
        if dist < min_dist: # and dist < max_distance_allowed:
            min_dist = dist
            winner = orow['geohash']                
    if winner is None:
        bad_count += 1
        print (f"No candidate found for {row['description']}\n {(row.geometry.y, row.geometry.x)}")
        print(f"Closest Distance: {min_dist_found}\n {(not_chosen_winner.geometry[1], not_chosen_winner.geometry[0])}")   
        print("\n")
    geohash_transform[row['geohash']] = winner
print(bad_count)
del bad_count
del tdf
del unmatched_df

0


We now convert the geohashes in the crime dataframes and export

In [20]:
tdf = crime_df.copy()
tdf['geohash'] = tdf['geohash'].apply(lambda x: geohash_transform[x] if x in geohash_transform.keys() else x)
data_df = pd.merge(tdf, intersections_info_df, how='left', on=['geohash'])
non_matches = data_df[(data_df['street1'].isna()) & data_df['street2'].isna() & (data_df['street_names'].isna())]
print("Non Matches: " + str(non_matches.count()[0]))
non_matches.head()

Non Matches: 0


,objectid,case_no,date,description,location,city,latitude,longitude,globalid,geohash,...,street_types,owners,max_width,min_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes,max_width_per_lane,min_width_per_lane


In [21]:
data_df.to_json('prepared_data/prepared_data.json', orient='records')
data_df.head()

,objectid,case_no,date,description,location,city,latitude,longitude,globalid,geohash,...,street_types,owners,max_width,min_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes,max_width_per_lane,min_width_per_lane
0,3,202200349491,"Fri, 25 Nov 2022 21:33:48 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.350226397496606,-79.80152666827507,f623ed61-fbc7-4391-8038-9e385d6ef3dd,hcyf42wd,...,"[MAJOR, MINOR]","[COB, ROH]",42.0,20.0,60.0,50.0,8.0,5.0,6.400000,3.333333
1,10,202200351471,"Sun, 27 Nov 2022 23:46:00 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.33830850583127,-79.7853229172281,aff64172-9cc4-44b0-a78b-71d0c21177c8,hcyf45tb,...,[MAJOR],[COB],35.0,24.0,60.0,50.0,6.0,5.0,7.000000,4.000000
2,13,202200353214,"Tue, 29 Nov 2022 15:40:00 GMT",MVC - HIT & RUN,GUELPH LI,BURLINGTON,43.3646561940604,-79.8220414963383,8883b2d6-9c65-4f61-a25d-5e9e3d77d4d9,hcycfuc4,...,[MAJOR],"[ROH, COB]",42.0,40.0,60.0,60.0,6.0,5.0,8.400000,7.000000
3,21,202200357807,"Sun, 04 Dec 2022 17:44:48 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.34732873413244,-79.79743701462455,48573cd9-b167-4c66-8bc7-7d6e32f546ee,hcyf43kd,...,[MAJOR],"[ROH, COB]",42.0,40.0,60.0,50.0,9.0,7.0,5.714286,4.444444
4,24,202200361004,"Thu, 08 Dec 2022 00:04:00 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.352177632465434,-79.80386318857246,6c97ac07-1a63-41b5-95f8-725dea1811f4,hcyf42pv,...,[MAJOR],"[ROH, MTO]",42.0,12.0,60.0,50.0,9.0,0.0,inf,4.666667
